In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect('../data/final_data.db')
cursor = conn.cursor()

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_names = [table[0] for table in tables]
print('Tables in the database:', table_names)

if len(table_names) == 2:
    print('Confirmed: Database contains exactly 2 tables')
else:
    print(f'Expected 2 tables, but found {len(table_names)} tables.')

for table_name in table_names:
    print(f"\nContents of table '{table_name}':")
    query = f'SELECT * FROM {table_name} LIMIT 5'
    df =  pd.read_sql_query(query, conn)
    print(df)

Tables in the database: ['fatalities', 'vehicle_prices']
Confirmed: Database contains exactly 2 tables

Contents of table 'fatalities':
   Year  Passenger Cars  Light Trucks  Large Trucks  Motorcycles  Buses  \
0  2015           19810         18869          4075         5131    263   
1  2014           17895         17160          3749         4707    236   
2  2013           17957         16928          3921         4800    282   
3  2012           18269         17350          3825         5113    253   
4  2011           17508         16806          3633         4769    245   

   Other/Unknown  Total  
0           1330  49478  
1           1205  44950  
2           1213  45101  
3           1150  45960  
4           1158  44119  

Contents of table 'vehicle_prices':
   year   make                model        trim   body transmission  \
0  2015    Kia              Sorento          LX    SUV    automatic   
1  2015    Kia              Sorento          LX    SUV    automatic   
2  2014

Setting up a query funtion that will take a string

In [4]:
def query(query: str):
    return pd.read_sql(query, conn)

In [5]:
all = """SELECT * FROM fatalities LIMIT 5"""
query(all)

,Year,Passenger Cars,Light Trucks,Large Trucks,Motorcycles,Buses,Other/Unknown,Total
0,2015,19810,18869,4075,5131,263,1330,49478
1,2014,17895,17160,3749,4707,236,1205,44950
2,2013,17957,16928,3921,4800,282,1213,45101
3,2012,18269,17350,3825,5113,253,1150,45960
4,2011,17508,16806,3633,4769,245,1158,44119


In [6]:
more = """SELECT * FROM vehicle_prices LIMIT 5"""
query(more)

,year,make,model,trim,body,transmission,vin,state,mmr,sellingprice,body_class
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,20500.0,21500.0,Light Trucks
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,20800.0,21500.0,Light Trucks
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,31900.0,30000.0,Passenger Cars
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,27500.0,27750.0,Passenger Cars
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,66000.0,67000.0,Passenger Cars


In [9]:
join = """
SELECT *
FROM vehicle_prices
LEFT JOIN fatalities
ON fatalities.year = vehicle_prices.year;
"""
query(join)

,year,Light Trucks Sold,Passenger Cars Sold,Year,Passenger Cars,Light Trucks,Large Trucks,Motorcycles,Buses,Other/Unknown,Total
0,1994,43,243,1994,30273,16353,4644,2339,258,1044,54911
1,1995,92,391,1995,30940,17587,4472,2268,271,986,56524
2,1996,114,448,1996,30727,18246,4755,2176,325,1117,57347
3,1997,251,791,1997,30059,18628,4917,2160,297,999,57060
4,1998,322,1142,1998,29040,19363,4955,2334,289,941,56922
5,1999,652,1575,1999,28027,19959,4920,2532,319,1063,56820
6,2000,1021,2406,2000,27802,20498,4995,2975,325,998,57593
7,2001,1957,3183,2001,27586,20831,4823,3265,292,1121,57918
8,2002,3204,4489,2002,27374,21668,4587,3365,274,1160,58428
9,2003,4603,5765,2003,26562,22299,4721,3802,291,1202,58877
